In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Загрузка данных
data = pd.read_csv("/content/drive/MyDrive/Data/digit_data_train.csv")

In [ ]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # Перемешивание данных

# Нормализация и разделение данных
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n] / 255 # Нормализация пикселей

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n] / 255 # Нормализация пикселей

In [ ]:
# Инициализация параметров
def init_params():
  hidden_size = 128 # Увеличенный размер скрытого слоя
  W1 = np.random.randn(hidden_size, 784) * np.sqrt(2./784) # He инициализация
  b1 = np.zeros((hidden_size, 1)) # Нулевая инициализация смещений
  W2 = np.random.randn(10, hidden_size) * np.sqrt(2./hidden_size)
  b2 = np.zeros((10, 1))
  return W1, b1, W2, b2

# Функции активации
def ReLU(Z):
  return np.maximum(0, Z)

def softmax(Z):
  exp_Z = np.exp(Z - np.max(Z)) # Стабильный softmax
  return exp_Z / exp_Z.sum(axis=0, keepdims=True)

# Прямое распространение
def forward_prop(W1, b1, W2, b2, X):
  Z1 = W1.dot(X) + b1
  A1 = ReLU(Z1)
  Z2 = W2.dot(A1) + b2
  A2 = softmax(Z2)
  return Z1, A1, Z2, A2

# One-hot кодирование
def one_hot(Y):
  one_hot_Y = np.zeros((Y.size, Y.max() + 1))
  one_hot_Y[np.arange(Y.size), Y] = 1
  return one_hot_Y.T

# Производные активаций
def deriv_ReLU(Z):
  return Z > 0

# Обратное распространение
def back_prop(Z1, A1, Z2, A2, W2, X, Y, m):
  one_hot_Y = one_hot(Y)
  dZ2 = A2 - one_hot_Y
  dW2 = 1/m * dZ2.dot(A1.T)
  db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
  dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
  dW1 = 1/m * dZ1.dot(X.T)
  db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)
  return dW1, db1, dW2, db2

# Обновление параметров
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
  W1 -= alpha * dW1
  b1 -= alpha * db1
  W2 -= alpha * dW2
  b2 -= alpha * db2
  return W1, b1, W2, b2

In [ ]:
# Метрики
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.mean(predictions == Y)

# Основной цикл обучения
def gradient_descent(X, Y, dev_X, dev_Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    m = X.shape[1]

    for i in range(iterations):
        # Прямое распространение
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)

        # Обратное распространение
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y, m)

        # Обновление параметров
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        # Логирование каждые 100 итераций
        if i % 100 == 0:
            # Точность на тренировочных данных
            train_pred = get_predictions(A2)
            train_acc = get_accuracy(train_pred, Y)

            # Точность на валидационных данных
            _, _, _, dev_A2 = forward_prop(W1, b1, W2, b2, dev_X)
            dev_pred = get_predictions(dev_A2)
            dev_acc = get_accuracy(dev_pred, dev_Y)

            print(f"Iteration: {i}")
            print(f"Train Accuracy: {train_acc:.3f} | Dev Accuracy: {dev_acc:.3f}")

    return W1, b1, W2, b2

In [ ]:
# Запуск обучения
W1, b1, W2, b2 = gradient_descent(
    X_train, Y_train,
    X_dev, Y_dev,
    iterations=1000,
    alpha=0.1
)

Iteration: 0
Train Accuracy: 0.097 | Dev Accuracy: 0.171
Iteration: 100
Train Accuracy: 0.882 | Dev Accuracy: 0.871
Iteration: 200
Train Accuracy: 0.903 | Dev Accuracy: 0.892
Iteration: 300
Train Accuracy: 0.912 | Dev Accuracy: 0.902
Iteration: 400
Train Accuracy: 0.919 | Dev Accuracy: 0.911
Iteration: 500
Train Accuracy: 0.925 | Dev Accuracy: 0.917
Iteration: 600
Train Accuracy: 0.930 | Dev Accuracy: 0.924
Iteration: 700
Train Accuracy: 0.934 | Dev Accuracy: 0.931
Iteration: 800
Train Accuracy: 0.937 | Dev Accuracy: 0.936
Iteration: 900
Train Accuracy: 0.940 | Dev Accuracy: 0.939
